In [12]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [13]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE


### Text truncation
- check how many docs exceed max

In [14]:
import pandas as pd
df = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")

In [14]:
print(f"{len(df.loc[df['token_count']>4096])} out of {len(df)} ({round(len(df.loc[df['token_count']>4096])/len(df)*100, 2)}%) docs exceed max window length of mistral (4096) ")
print(f"{len(df.loc[df['token_count']>1000])} out of {len(df)} ({round(len(df.loc[df['token_count']>1000])/len(df)*100, 2)}%) docs are longer than 1000 tokens")





1964 out of 26704 (7.35%) docs exceed max window length of mistral (4096) 
7831 out of 26704 (29.33%) docs are longer than 1000 tokens


In [20]:
display(df.loc[df['token_count']>1000].describe().drop(columns=['id']))

,token_count,clean_tokens_count,num_pages
count,7831.000000,7831.000000,7831.000000
mean,5867.532499,2937.387435,13.957732
std,12054.449388,6257.314713,25.415472
min,1001.000000,439.000000,1.000000
25%,1305.000000,660.000000,4.000000
50%,1905.000000,954.000000,5.000000
75%,4128.000000,2089.500000,11.000000
max,275597.000000,143782.000000,502.000000


In [24]:
yeet = pd.read_pickle(f"{cf.output_path}/txtfiles_notcleaned.pkl")

In [26]:
txt = df.iloc[0]['text']

In [22]:
print(df.iloc[0]['token_count'])

2452


In [46]:
from transformers import AutoTokenizer

def get_token_length(model_name, df, save_to_path, text_col, new_col_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    all_texts = list(df[text_col].values)

    all_len_tokens = []
    for txt in all_texts:
        len_tokens = len(tokenizer.tokenize(txt))
        all_len_tokens.append(len_tokens)

    df[new_col_name] = all_len_tokens
    df.to_pickle(save_to_path)
    return df

# subdf = df.iloc[0:2]
# # display(subdf)
# get_token_length('Rijgersberg/GEITje-7B-chat-v2', subdf, f"{cf.output_path}/try_out_token_count.pkl", 'text', 'token_count_geitje')

def fraction_token(df, max_token, token_len_col):
    for col in token_len_col:
        print(f"{len(df.loc[df[col]>max_token])} out of {len(df)} ({round(len(df.loc[df[col]>max_token])/len(df)*100, 2)}%) docs exceed a token length of {max_token}")

    for col in token_len_col:
        print(df[col].describe())

    


    

### Get token lengths of model tokenizers

In [47]:
"""GEITje"""
# df = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")
# get_token_length('Rijgersberg/GEITje-7B-chat-v2', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'token_count_geitje')

"""Mistral"""
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_token_length('mistralai/Mistral-7B-v0.1', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'token_count_mistral')

"""Llama"""
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_token_length('meta-llama/Llama-2-7b-hf', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'token_count_llama2_7b_hf')

'Llama'

### Analyse token length of model tokenizers

In [51]:
df_token_len = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
fraction_token(df_token_len, 4096, ['token_count_geitje', 'token_count_mistral', 'token_count_llama2_7b_hf'])

4005 out of 26704 (15.0%) docs exceed a token length of 4096
4005 out of 26704 (15.0%) docs exceed a token length of 4096
3839 out of 26704 (14.38%) docs exceed a token length of 4096
count     26704.000000
mean       4310.098749
std       14728.324592
min          75.000000
25%         680.000000
50%        1213.000000
75%        2609.000000
max      621995.000000
Name: token_count_geitje, dtype: float64
count     26704.000000
mean       4310.098749
std       14728.324592
min          75.000000
25%         680.000000
50%        1213.000000
75%        2609.000000
max      621995.000000
Name: token_count_mistral, dtype: float64
count     26704.000000
mean       4167.322124
std       14243.284787
min          74.000000
25%         662.000000
50%        1184.000000
75%        2530.000000
max      618067.000000
Name: token_count_llama2_7b_hf, dtype: float64


### Mistral tokenizer

In [29]:
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_token_length('mistralai/Mistral-7B-v0.1', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'token_count_mistral')

# model_name = "mistralai/Mistral-7B-v0.1"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Tokenize some input.
# tokens = tokenizer.tokenize(txt)
# print(len(tokens))

In [ ]:
fraction_token(df_token_len, 1000, 'token_count_mistral')

### Llama tokenizer

In [31]:
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_token_length('meta-llama/Llama-2-7b-hf', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'token_count_llama2_7b_hf')


In [27]:
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize some input.
tokens = tokenizer.tokenize(txt)
print(len(tokens))

4924
